In [1]:
from sklearn import svm, preprocessing, metrics
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler

In [2]:
import pandas as pd
import numpy as np
import os
from collections import Counter
import random

In [3]:
def to_csv_helper(data, filename):
    if os.path.exists(filename):
        print('file already exists')
        return 0
    data.to_csv(filename, index=False)
    return 0

In [4]:
#a = all_data[0]
# a.head()

In [5]:
# a.columns

In [6]:
def AU(n):
    # return AU column name
    return f'AU{str(n).zfill(2)}_r'

In [7]:
# ex = all_data[0]
# # for x in range(ex['milis'].max):
# #     print(i)

# x = 1
# windows = []
# while x + 3000 < max(ex['milis']):
#     windows.append((x, x + 3000))
#     x += 3010

# print(windows)
# len(windows)

In [8]:
# define useful functions

count = 0
RANGE = 1500
# RANGE = 3000
def closest_frame(df, f):
    f_near = f
    for x in range(50):
        while len(face_data.loc[face_data['frame'] == f_near, 'milis'].values) == 0:
            f_near -= 1
#     print("number of frames retraced: " + str(f - f_near))
    return f_near

def df_near(df, frame, forwards=False, ms_range=RANGE):
#     global count
#     count += len(frames_list)
    if frame is None:
        return None
    ms = df.loc[df['frame'] == closest_frame(df,frame), 'milis'].values[0]
    if forwards:
        locs = df.loc[(df['milis'] >= ms - ms_range) & (df['milis'] <= ms + ms_range)]
    else:
        locs = df.loc[(df['milis'] >= ms - ms_range) & (df['milis'] <= ms)]
    return locs.dropna()

def df_near_tuple(df, milis, forwards=False, ms_range=RANGE):
    if forwards:
        ms_range = ms_range / 2
    locs = df.loc[(df['milis'] >= milis[0]) & (df['milis'] <= milis[1])]
    return locs.dropna()

def max_df_near(df, frames_list, ms_range=RANGE):
    d = df_near(df, frames_list)
    if d is None:
        return None
    return pd.DataFrame(d.max()).transpose()

def min_df_near(df, frames_list, ms_range=RANGE):
    d = df_near(df, frames_list)
    if d is None:
        return None
    return pd.DataFrame(d.min()).transpose()

def max_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.max()).transpose()

def min_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.min()).transpose()

def mean_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.mean()).transpose()

def sd_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.std()).transpose()

In [9]:
def overlap(a,b):
    return a[0] <= b[0] <= a[1] or b[0] <= a[0] <= b[1]

In [10]:
# gather data

# choose which action units to look at
# AUs_to_look_at = [1,9,10,14,15,17,20,23]
Excluded_p = ['P6109bd72fa4cdd8f9585cc43','P60e7a66b2cb32cc90fade4e6']

def get_pid(filename):
    """ get pid from dirname """
    return filename.split('_')[0][1:]
def get_mode(filename):
    """ get mode from dirname """
    return int(file.split('_')[2][1])

up_data_max = []
down_data_max = []
up_data_min = []
down_data_min = []
all_data = []

up_data_sd = []
down_data_sd = []

up_data_mean = []
down_data_mean = []

neutral_data_max = []
neutral_data_min = []
neutral_data_mean = []
neutral_data_sd = []

all_data_max = []
all_data_min = []
all_data_mean = []
all_data_sd = []

PATH = "/Users/yoony/Desktop/si_research/face_analysis/implicit_feedback_si_faceanalysis/processed_merged"
signals = pd.read_csv('../all_pilots/signals_all_pilots.csv')[['player_id', 'mode', 'signal_type', 'frame_number']]

for file in sorted(os.listdir(PATH)):
    if file[0] == '.' or file.split('_')[0] in Excluded_p:
        print('excluded:' + file)
        continue
    print(file)
#     csvs.append(pd.read_csv(os.path.join(PATH + "/" + file, 'webcam/aggregated_webcam.csv')))
    data = pd.read_csv(os.path.join(PATH + "/" + file, 'webcam/aggregated_webcam.csv'))
    face_data = data.drop(columns=list(data.columns.values[2:678]))
    face_data = face_data.drop(columns=list(face_data.columns.values[19:]))
#     face_data = face_data[(['frame', 'milis'] + [AU(x) for x in AUs_to_look_at])]
    
    s1 = signals.loc[(signals['player_id'] == get_pid(file)) & (signals['mode'] == get_mode(file))]
    s_up = s1.loc[s1['signal_type'] == 'up'].frame_number.values
    s_down = s1.loc[s1['signal_type'] == 'down'].frame_number.values
    
    covered_windows = []
    
    for x in s_up:
        up_locs = df_near(face_data, x, forwards=False)
        up_data_max.append(max_df(up_locs))
        up_data_min.append(min_df(up_locs))
        up_data_sd.append(sd_df(up_locs))
        up_data_mean.append(mean_df(up_locs))
        
#         all_data_max.append(max_df(up_locs))
#         all_data_min.append(min_df(up_locs))
#         all_data_sd.append(sd_df(up_locs))
#         all_data_mean.append(mean_df(up_locs))
        
        covered_windows.append((min(up_locs['milis']), max(up_locs['milis'])))
        
    for x in s_down:
        down_locs = df_near(face_data, x, forwards=False)
        down_data_max.append(max_df(down_locs))
        down_data_min.append(min_df(down_locs))
        down_data_sd.append(sd_df(down_locs))
        down_data_mean.append(mean_df(down_locs))
        
#         all_data_max.append(max_df(down_locs))
#         all_data_min.append(min_df(down_locs))
#         all_data_sd.append(sd_df(down_locs))
#         all_data_mean.append(mean_df(down_locs))
        
        covered_windows.append((min(down_locs['milis']), max(down_locs['milis'])))
    
    # Get non-overlapping neutral (no signal) windows
    windows = []
    # start from when recording starts
    i = min(face_data['milis'])
    while i + RANGE < max(face_data['milis']):
        w = (i, i+(RANGE))
        clash = False
        for c in covered_windows:
            if overlap(w,c):
                clash = True
                i = c[1] + 100
        if not clash:
            windows.append(w)
            i += 1510
            
#     print("stat" + str(max(face_data['milis'] / 3000)))
#     print(covered_windows)
#     print(len(windows))
    for w in windows:
        neutral_locs = df_near_tuple(face_data, w, forwards=False)
        neutral_data_max.append(max_df(neutral_locs))
        neutral_data_min.append(min_df(neutral_locs))
        neutral_data_sd.append(sd_df(neutral_locs))
        neutral_data_mean.append(mean_df(neutral_locs))
        
#         all_data_max.append(max_df(neutral_locs))
#         all_data_min.append(min_df(neutral_locs))
#         all_data_sd.append(sd_df(neutral_locs))
#         all_data_mean.append(mean_df(neutral_locs))
    
#     print(sorted(covered_windows + windows))

        
    all_data.append(face_data)
    
    

excluded:.DS_Store
P5ee6d2494fd04045b988a12d_voff_m1_g1_t2021_7_10_0_47
P5ee6d2494fd04045b988a12d_voff_m2_g2_t2021_7_10_0_50
P5ef33c05f4c5240413ef9ac2_voff_m1_g1_t2021_7_13_20_38
P5ef33c05f4c5240413ef9ac2_voff_m2_g2_t2021_7_13_20_34
P5fc9a415aacaf14df94e6868_voff_m1_g1_t2021_8_4_20_48
P5fc9a415aacaf14df94e6868_voff_m2_g2_t2021_8_4_20_52
P6081c511ded5945d9b70801e_voff_m1_g1_t2021_7_13_21_6
P6081c511ded5945d9b70801e_voff_m2_g2_t2021_7_13_21_11
P60d520539b147c4acaf44b77_voff_m1_g1_t2021_7_10_1_4
P60d520539b147c4acaf44b77_voff_m2_g2_t2021_7_10_1_1
P60e5e62659dc1912bc43ed35_voff_m1_g1_t2021_7_9_23_29
P60e5e62659dc1912bc43ed35_voff_m2_g2_t2021_7_9_23_24
excluded:P60e7a66b2cb32cc90fade4e6_voff_m1_g1_t2021_7_10_0_45
excluded:P60e7a66b2cb32cc90fade4e6_voff_m2_g2_t2021_7_10_0_49
P60e9e9618953d7a3430dd8f6_voff_m1_g1_t2021_7_13_18_51
P60e9e9618953d7a3430dd8f6_voff_m2_g2_t2021_7_13_18_57
P60edb7015a9231789057bdea_voff_m1_g1_t2021_7_13_21_7
P60edb7015a9231789057bdea_voff_m2_g2_t2021_7_13_21_11
P60fd

In [11]:
print(len(neutral_data_max))
print(len(up_data_max))
print(len(down_data_max))

1837
249
69


In [12]:
def rename_columns(df, tag):
    a = df.copy()
    a.columns = list(map(lambda x: x + tag, a.columns))
    return a

def merge_dfs(dfs):
    a = dfs[0]
    for d in dfs[1:]:
        a = a.join(d, how='outer')
    return a
        

In [13]:
u1 = rename_columns(pd.concat(up_data_max).drop(columns=['frame', 'milis']), '_max').reset_index(drop=True)
u2 = rename_columns(pd.concat(up_data_min).drop(columns=['frame', 'milis']), '_min').reset_index(drop=True)
u3 = rename_columns(pd.concat(up_data_mean).drop(columns=['frame', 'milis']), '_mean').reset_index(drop=True)
u4 = rename_columns(pd.concat(up_data_sd).drop(columns=['frame', 'milis']), '_sd').reset_index(drop=True)

In [14]:
# ups = merge_dfs([u1, u2, u3, u4])
ups = merge_dfs([u1, u2, u3, u4])
ups['type'] = 'up'

# ups['type'] = 'signal'

In [15]:
d1 = rename_columns(pd.concat(down_data_max).drop(columns=['frame', 'milis']), '_max').reset_index(drop=True)
d2 = rename_columns(pd.concat(down_data_min).drop(columns=['frame', 'milis']), '_min').reset_index(drop=True)
d3 = rename_columns(pd.concat(down_data_mean).drop(columns=['frame', 'milis']), '_mean').reset_index(drop=True)
d4 = rename_columns(pd.concat(down_data_sd).drop(columns=['frame', 'milis']), '_sd').reset_index(drop=True)

In [16]:
# downs = merge_dfs([d1, d2, d3, d4])
downs = merge_dfs([d1, d2, d3, d4])
downs['type'] = 'down'
# downs['type'] = 'signal'

In [17]:
n1 = rename_columns(pd.concat(neutral_data_max).drop(columns=['frame', 'milis']), '_max').reset_index(drop=True)
n2 = rename_columns(pd.concat(neutral_data_min).drop(columns=['frame', 'milis']), '_min').reset_index(drop=True)
n3 = rename_columns(pd.concat(neutral_data_mean).drop(columns=['frame', 'milis']), '_mean').reset_index(drop=True)
n4 = rename_columns(pd.concat(neutral_data_sd).drop(columns=['frame', 'milis']), '_sd').reset_index(drop=True)

In [18]:
neutrals = merge_dfs([n1,n2,n3,n4])
neutrals['type'] = 'neutral'

In [19]:
means = pd.DataFrame(pd.concat(up_data_max).mean(), columns=['up_mean'])
means['down_mean'] = pd.concat(down_data_max).mean()
means['mean_diff'] = means['up_mean'] - means['down_mean']
means['neutral_mean'] = pd.concat(neutral_data_max).mean()
means['up_sd'] = pd.concat(up_data_sd).std()
means['down_sd'] = pd.concat(down_data_sd).std()
means['sd_diff'] = means['up_sd'] - means['down_sd']
means = means.drop(index=['frame', 'milis'])
# means['total'] = pd.concat(all_data).mean
# means.to_csv('face/au_data.csv')
means

,up_mean,down_mean,mean_diff,neutral_mean,up_sd,down_sd,sd_diff
AU01_r,1.178996,1.019855,0.159141,1.211399,0.082051,0.079983,0.002068
AU02_r,0.888635,0.915797,-0.027163,0.973751,0.094342,0.123306,-0.028964
AU04_r,0.812610,0.684203,0.128408,0.678149,0.080020,0.096669,-0.016649
AU05_r,0.827349,0.818841,0.008509,0.795934,0.118106,0.163830,-0.045724
AU06_r,0.135261,0.101739,0.033522,0.144747,0.061646,0.077680,-0.016034
AU07_r,0.439076,0.280870,0.158207,0.456173,0.119859,0.106249,0.013610
AU09_r,0.396145,0.411014,-0.014870,0.412014,0.076972,0.079187,-0.002215
AU10_r,0.209839,0.309710,-0.099871,0.308051,0.093079,0.096286,-0.003207
AU12_r,0.118394,0.098696,0.019698,0.160953,0.054819,0.054444,0.000375
AU14_r,0.358554,0.314203,0.044351,0.365264,0.095043,0.101816,-0.006772


In [20]:
# check number of signals
len(up_data_max) + len(down_data_max) + len(neutral_data_max)

2155

In [21]:
len(neutrals)

1837

In [22]:
#clean up and finalize the data
signal_data = pd.concat([ups, downs, neutrals]).reset_index(drop=True)
signal_data = signal_data.dropna()

to_csv_helper(signal_data, f'face_data/all_pilots/three_classes/fd_oneside_{RANGE}.csv')
# signal_data.to_csv(f'face_data/three/fd_2c_oneside_no45_{RANGE}.csv', index=False)

0

In [23]:
Counter(signal_data['type'])

Counter({'up': 249, 'down': 69, 'neutral': 1837})